In [2]:
import warnings 
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import itertools

# Ignorando warnings 

warnings.filterwarnings('ignore')

In [3]:
# Carregando os dados

df_train = pd.read_csv('application_train.csv')
df_test = pd.read_csv('application_test.csv')

In [12]:
df_train.sort_values(by=['TARGET'], ascending=False).head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
142676,265437,1,Cash loans,M,N,N,0,76500.0,495000.0,16096.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
71932,183409,1,Cash loans,F,N,Y,0,225000.0,247275.0,22810.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0
58253,167527,1,Cash loans,M,Y,Y,3,150750.0,315000.0,34056.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
142665,265426,1,Cash loans,M,Y,Y,2,225000.0,473760.0,50400.0,...,0,0,0,0,0.0,0.0,0.0,1.0,1.0,2.0


In [4]:
# Dropando coluna inútil

df_train.drop(["SK_ID_CURR"], axis=1, inplace=True)
df_test.drop(["SK_ID_CURR"], axis=1, inplace=True)

In [5]:
df_test.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Filtrando colunas não numéricas

df_train.select_dtypes(exclude=np.number).columns

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
      dtype='object')

In [7]:
# Criando cópia do dataset para tratar valores nulos

df_temp = df_train.copy()

In [8]:
# Dropando colunas não numéricas

df_temp.drop(df_temp.select_dtypes(exclude=np.number).columns, axis=1, inplace=True)

In [9]:
df_temp

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327,-236,-8456.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,0,0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775,365243,-4388.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,0,0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966,-7921,-6737.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,1,0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961,-4786,-2562.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Criando DataFrame com o nome das features, valores nulos e desvio padrão

df_null_values = pd.DataFrame({'columns': df_temp.columns,
                             'number_of_nulls': df_temp.isna().sum(),
                             'standard_deviation': df_temp.std()
                              }).reset_index(drop=True)
df_null_values

,columns,number_of_nulls,standard_deviation
0,TARGET,0,0.272419
1,CNT_CHILDREN,0,0.722121
2,AMT_INCOME_TOTAL,0,237123.146279
3,AMT_CREDIT,0,402490.776996
4,AMT_ANNUITY,12,14493.737315
...,...,...,...
100,AMT_REQ_CREDIT_BUREAU_DAY,41519,0.110757
101,AMT_REQ_CREDIT_BUREAU_WEEK,41519,0.204685
102,AMT_REQ_CREDIT_BUREAU_MON,41519,0.916002
103,AMT_REQ_CREDIT_BUREAU_QRT,41519,0.794056


In [11]:
# Criando coluna com o percentual de nulos

df_null_values['percentage'] = df_null_values.number_of_nulls / df_temp.shape[0]

In [12]:
# Dropando colunas que não possuem valores nulos

df_null_values.drop(df_null_values.loc[df_null_values['number_of_nulls'] == 0].index, inplace=True)

In [13]:
# Sort da coluna 'standard_deviation' em ordem decrescente

df_null_values.sort_values(by='standard_deviation', ascending=False)

,columns,number_of_nulls,standard_deviation,percentage
5,AMT_GOODS_PRICE,278,369446.460540,0.000904
4,AMT_ANNUITY,12,14493.737315,0.000039
78,DAYS_LAST_PHONE_CHANGE,1,826.808487,0.000003
11,OWN_CAR_AGE,202929,11.944812,0.659908
74,OBS_30_CNT_SOCIAL_CIRCLE,1021,2.400989,0.003320
...,...,...,...,...
61,YEARS_BEGINEXPLUATATION_MEDI,150007,0.059897,0.487810
33,YEARS_BEGINEXPLUATATION_AVG,150007,0.059223,0.487810
43,NONLIVINGAPARTMENTS_AVG,213514,0.047732,0.694330
71,NONLIVINGAPARTMENTS_MEDI,213514,0.047415,0.694330


In [14]:
# Filtrando features com uma quantidade de valores nulos maior que 15%

df_null_values = df_null_values.loc[df_null_values['percentage'] >= 0.15]
df_null_values

,columns,number_of_nulls,standard_deviation,percentage
11,OWN_CAR_AGE,202929,11.944812,0.659908
28,EXT_SOURCE_1,173378,0.211062,0.563811
30,EXT_SOURCE_3,60965,0.194844,0.198253
31,APARTMENTS_AVG,156061,0.108240,0.507497
32,BASEMENTAREA_AVG,179943,0.082438,0.585160
33,YEARS_BEGINEXPLUATATION_AVG,150007,0.059223,0.487810
34,YEARS_BUILD_AVG,204488,0.113280,0.664978
35,COMMONAREA_AVG,214865,0.076036,0.698723
36,ELEVATORS_AVG,163891,0.134576,0.532960
37,ENTRANCES_AVG,154828,0.100049,0.503488


In [15]:
# Analisando o desvio padrão após o filtro

df_null_values.standard_deviation.describe()

count    46.000000
mean      0.361173
std       1.746262
min       0.046276
25%       0.077404
50%       0.100673
75%       0.127512
max      11.944812
Name: standard_deviation, dtype: float64

In [16]:
# Dropando features com um desvio padrão abaixo da mediana

df_null_values = df_null_values.loc[df_null_values['standard_deviation'] < 0.10]
df_null_values

,columns,number_of_nulls,standard_deviation,percentage
32,BASEMENTAREA_AVG,179943,0.082438,0.585160
33,YEARS_BEGINEXPLUATATION_AVG,150007,0.059223,0.487810
35,COMMONAREA_AVG,214865,0.076036,0.698723
40,LANDAREA_AVG,182590,0.081184,0.593767
41,LIVINGAPARTMENTS_AVG,210199,0.092576,0.683550
43,NONLIVINGAPARTMENTS_AVG,213514,0.047732,0.694330
44,NONLIVINGAREA_AVG,169682,0.069523,0.551792
46,BASEMENTAREA_MODE,179943,0.084307,0.585160
47,YEARS_BEGINEXPLUATATION_MODE,150007,0.064575,0.487810
49,COMMONAREA_MODE,214865,0.074445,0.698723


In [17]:
Q1 = df_train.quantile(0.25)
Q3 = df_train.quantile(0.75)
IQR = Q3 - Q1
df_train = df_train[~((df_train < (Q1- 1.5* IQR)) | (df_train > (Q3 + 1.5 * IQR))).any(axis=1)]
df_train.shape

(26808, 121)

In [18]:
# Checando as linhas duplicadas
df_train.duplicated().sum()

0

In [19]:
# checando colunas duplicadas

# listar combinações das colunas em pares
comb = list(itertools.combinations(df_train.columns, 2))

# iterar pelas combinações de coluna e comparar os pares
dup_cols = []
for col1, col2 in comb:
    if ((col1 not in dup_cols) & (col2 not in dup_cols)):
        if df_train[col1].equals(df_train[col2]):   # aqui ocorre a comparação
            dup_cols.append(col1)

print(f'Quantidade de Colunas Duplicadas: {len(dup_cols)}')

Quantidade de Colunas Duplicadas: 35


In [20]:
# Inserindo a mediana no lugar dos valores nulos das features numéricas com SimpleImputer

imputer = SimpleImputer(strategy='median', missing_values=np.nan)
imputer = imputer.fit(df_train[null_numeric_features])
df_train[null_numeric_features] = imputer.transform(df_train[null_numeric_features])

NameError: name 'null_numeric_features' is not defined